In [ ]:
from lstm_model import build_open_loop_lstm, load_open_loop_lstm
from data_processing import create_training_split, df_training_split, create_df_3d, create_window_closed_loop, add_new_pred, compute_lyapunov_time_arr, train_valid_test_split, df_train_valid_test_split
import warnings
import seaborn as sns
import datetime
import random
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import tensorflow_datasets as tfds
import plots
import importlib
plt.rcParams['figure.facecolor'] = 'w'
warnings.simplefilter(action='ignore', category=FutureWarning)

np.random.seed(1234)
tf.random.set_seed(1234)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%rm -rf ./50_window_100LSTM/Images/LSTM_0_washout/run_1502/logs/fit/250_oloop/

In [ ]:
mydf = np.genfromtxt('CSV/lorenz_normalized.csv', delimiter=',')
time = mydf[0, :]
mydf = mydf[1:, :]
df_train, df_valid, df_test = df_train_valid_test_split(mydf)
time_train, time_valid, time_test = train_valid_test_split(time)
x_train, x_valid, x_test = train_valid_test_split(mydf[0, :])
y_train, y_valid, y_test = train_valid_test_split(mydf[1, :])
z_train, z_valid, z_test = train_valid_test_split(mydf[2, :])

In [ ]:
window_size = 50
batch_size = 32
shuffle_buffer_size = df_train.shape[0]
train_dataset = create_df_3d(df_train.transpose(
), window_size, batch_size, shuffle_buffer_size)
valid_dataset = create_df_3d(df_valid.transpose(), window_size, batch_size, 1)
test_dataset = create_df_3d(df_test.transpose(), window_size, batch_size, 1)

In [ ]:
def custom_loss(y_true, y_pred, washout=0):
    mse = tf.keras.losses.MeanSquaredError()
    # (batchsize, dimensions)
    loss = mse(y_true[washout:, :], y_pred[washout:, :])
    return loss

In [ ]:
def build_open_loop_lstm():
    lstm_init = tf.keras.initializers.GlorotUniform(seed=0)
    dense_init = tf.keras.initializers.GlorotUniform(seed=1)
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(8, activation='tanh',
                             name='LSTM_1', kernel_initializer=lstm_init),
        tf.keras.layers.Dense(3, name='Dense_1', kernel_initializer=dense_init)
    ])
    optimizer = tf.keras.optimizers.Adam()
    mse_loss = tf.keras.losses.MeanSquaredError()
    model.compile(loss=custom_loss, optimizer=optimizer, metrics=['mse'])
    return model

In [ ]:
def random_int_window_dist(df_transposed, n_int, window_size=50, batch_size=32):
    idx = [batch_size * i for i in sorted(random.sample(range(185), n_int))]
    return idx


def add_cloop_prediction(df_transposed, idx, predictions):
    window_list = np.array(
        [
            np.append(
                df_transposed[idx[i] + 1 : idx[i] + 50, :],
                predictions[i, :].reshape(1, 3),
                axis=0,
            ).reshape(50, 3)
            for i in range(0, len(idx))
        ]
    )
    label_list = np.array([df_transposed[i + 52, :] for i in idx])
    return window_list, label_list

In [ ]:
model_oloop.optimizer.lr.assign(new_learning_rate)

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
model_oloop = build_open_loop_lstm()
log_dir = "50_window_100LSTM/Images/LSTM_0_washout/run_1602/logs/fit/" + \
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss', patience=10, restore_best_weights=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)

history = model_oloop.fit(train_dataset, epochs=100, batch_size=batch_size,
                          validation_data=valid_dataset, verbose=1, callbacks=[tensorboard_callback])

img_filepath = '50_window_100LSTM/Images/LSTM_10_washout/run_1602/'
if not os.path.exists(img_filepath):
    os.makedirs(img_filepath)
lya_filepath = img_filepath + 'cloop_' + str(history.params['epochs'])+'.png'
predictions = plots.plot_closed_loop_lya(
    model_oloop, history.params['epochs'], time_test, df_test, n_length=1950)
phase_filepath = img_filepath + 'phase_cloop_' + str(history.params['epochs'])+'.png'
plots.plot_phase_space(
    predictions, history.params['epochs'], df_test)

In [ ]:
cloop_size = 32*1
test_window, labels, idx = select_random_batch_with_label(
    df_train.transpose())#, cloop_size)  # np.append(df_train[:, -50:], df_test, axis=1)
predictions = model.predict(np.array(test_window).reshape(cloop_size, 50, 3))

print(predictions.shape)
cloop_windows, cloop_label = add_cloop_prediction(
    df_train.transpose(), idx, predictions)
print(np.array(cloop_windows).shape, cloop_label.shape)


In [ ]:
for i in range(0, 5):
    print('Iteration is ', i)
    cloop_size = 32*1
    test_window, labels, idx = select_random_batch_with_label(
    df_train.transpose())#, cloop_size)  # np.append(df_train[:, -50:], df_test, axis=1)
    predictions = model.predict(np.array(test_window).reshape(cloop_size, 50, 3))
    cloop_windows, cloop_label = add_cloop_prediction(
    df_train.transpose(), idx, predictions)
    history = model.fit(cloop_windows, cloop_label, epochs=100, batch_size=32,
                        validation_data=valid_dataset, callbacks=[tensorboard_callback, early_stop_callback], verbose=2)

    predictions = plots.plot_closed_loop_lya(
        model, history.params['epochs'], time_test, df_test, n_length=1950)

    plots.plot_phase_space(
        predictions, history.params['epochs'], df_test)

In [ ]:
img_filepath = '100_window_200LSTM/Images/LSTM_10_washout/run_1002/'
lya_filepath = img_filepath + 'cloop_' + str(history.params['epochs'])+'.png'
predictions = plots.plot_closed_loop_lya(
    model, history.params['epochs'], time_test, df_test, n_length=400)
phase_filepath = img_filepath + 'phase_cloop_' + \
    str(history.params['epochs'])+'.png'
plots.plot_phase_space(
    predictions, history.params['epochs'], df_test)

In [ ]:

img_filepath = '50_window_100LSTM/Images/LSTM_10_washout/run_1002/'

if not os.path.exists(img_filepath):
    os.makedirs(img_filepath)

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
model_oloop = build_open_loop_lstm()
img_filepath = '100_window_200_LSTM_unnorm/Images/LSTM_10_washout/run_1002/'

n_epochs_old = 0
# Save the weights
for i in range(0, 1):
    n_epochs = n_epochs_old + 50
    history = model_oloop.fit(train_dataset, epochs=n_epochs, initial_epoch=n_epochs_old, batch_size=batch_size,
                              validation_data=valid_dataset, verbose=2)  # , callbacks=[tensorboard_callback]) #, early_stop_callback])
    n_epochs_old = n_epochs
    lya_filepath = img_filepath + 'cloop_' + str(n_epochs)+'.png'
    predictions = plots.plot_closed_loop_lya(
        model_oloop, n_epochs, time_test, df_test, lya_filepath, n_length=400)
    phase_filepath = img_filepath + 'phase_cloop_' + str(n_epochs)+'.png'
    plots.plot_phase_space(predictions, n_epochs, df_test, phase_filepath)

In [ ]:
predictions

In [ ]:
plots.plot_phase_space(predictions, n_epochs, df_test, phase_filepath)

In [ ]:
predictions = plots.plot_closed_loop_lya(
    model_oloop, n_epochs, time_test, df_test, lya_filepath, n_length=1900)
phase_filepath = img_filepath + 'phase_cloop_' + str(n_epochs)+'.png'
plots.plot_phase_space(predictions, n_epochs, df_test, phase_filepath)

In [ ]:
model = model_oloop
forecast = model.predict(test_dataset)

In [ ]:
err = df_test.transpose()[50:, :] - forecast
abs_error = np.abs(df_test.transpose()[50:, :] - forecast)

fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True,
                                    sharey=True, facecolor='w')  # , figsize=(15, 15))
fig.suptitle('Absolute Error on Test Data')
ax1.plot(time_test[50:], abs_error[:, 0])
ax1.set_ylabel(r'$|x_{true}-x_{pred}|$')
ax1.set_yscale('log')
ax2.plot(time_test[50:], abs_error[:, 1])
ax2.set_ylabel(r'$|y_{true}-y_{pred}|$')
ax2.set_yscale('log')
ax3.plot(time_test[50:], abs_error[:, 2])
ax3.set_ylabel(r'$|z_{true}-z_{pred}|$')
ax3.set_xlabel('t')
ax3.set_yscale('log')
ax3.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#fig.savefig(error_filepath, dpi=1000, facecolor="w")

In [ ]:
model_checkpoint = 'oloop_lstm_Washout_0/250_epoch_cp'
# Save the weights
model_oloop.save_weights(model_checkpoint)

In [ ]:
def select_random_window_with_label(df_transposed, n_windows, window_size=50, n_dim=3):
    idx = random.sample(range(len(df_transposed)-window_size-1), n_windows)
    window_list = np.array([df_transposed[i:i+window_size, :].reshape(window_size, 3)
                           for i in idx])  # .reshape((n_windows,window_size,n_dim))
    label_list = np.array([df_transposed[i+window_size+1, :].reshape(n_dim)
                          for i in idx])  # .reshape(n_windows, n_dim)
    return window_list, label_list, idx


def add_cloop_prediction(df_transposed, idx, predictions):
    window_list = np.array([np.append(df_train.transpose()[idx[i]+1:idx[i]+50, :],
                           predictions[i, :].reshape(1, 3), axis=0).reshape(50, 3) for i in range(0, len(idx))])
    label_list = np.array([df_transposed[i+52, :] for i in idx])
    return window_list, label_list

In [ ]:
def select_random_batch_with_label(df_transposed):
    idx_start = random.randint(0, len(df_transposed) - 50 - 1)
    idx = np.arange(start=idx_start, stop=idx_start + 32)
    # window_list =[]
    window_list = [df_transposed[i : i + 50, :].reshape(1, 50, 3) for i in idx]
    label_list = [df_transposed[i + 51, :].reshape(1, 3) for i in idx]
    return window_list, label_list, idx

In [ ]:
def random_int_window_dist(df_transposed, n_int, window_size=50, batch_size=32):
    """returns a list of random ints of length n_int which all have distance n_window"""
    idx = [
        (50+batch_size) * i + x
        for i, x in enumerate(
            sorted(random.sample(range(len(df_transposed) - window_size - batch_size - 1), n_int))
        )
    ]
    return idx

In [ ]:
def select_random_batches_with_label(df_transposed, n_int, window_size=50, batch_size=32):
    idx = random_int_window_dist(df_transposed, n_int, window_size=window_size)
    idx_list = np.array([np.arange(start=idx_start, stop=idx_start + 32) for idx_start in idx]).flatten()
    window_list = [df_transposed[i : i + 50, :].reshape(1, 50, 3) for i in idx_list]
    label_list = [df_transposed[i + 51, :].reshape(1, 3) for i in idx_list]
    return np.array(window_list), np.array(label_list), idx_list

In [ ]:
model_oloop.fit(window_list.reshape(64, 50, 3), label_list, epochs=2, batch_size=32,
                        validation_data=valid_dataset, callbacks=[tensorboard_callback, early_stop_callback], verbose=2)


In [ ]:
forecast = model.predict(train_dataset)
forecast_err = np.abs(df_train.transpose()[50:, :] - forecast)
plt.plot(forecast_err[:, 0])
plt.yscale('log')

In [ ]:
error = np.abs(labels - predictions)
plt.plot(error[:, 1])
plt.yscale('log')

In [ ]:
[50*i + x for i, x in enumerate(sorted(random.sample(range(1000), 10)))]

In [ ]:
[50*i + x for i, x in enumerate(sorted(random.sample(range(5), 5)))]